## Computing Gradients using AutoDiff

This section is to understand how to compute gradients automatically using tensorflow

In [23]:
from tensorflow import keras
import tensorflow as tf

In [24]:
def f(w1, w2):
    return 3 * w1 ** 2 + 2 * w1 * w2

# of course we can find the derivative at specific points by approximating using a small delta
w1, w2 = 5, 3
eps = 1e-6
(f(w1 + eps, w2) - f(w1, w2)) / eps, (f(w1, w2 + eps) - f(w1, w2)) / eps

(36.000003007075065, 10.000000003174137)

In [25]:
## Using tensorflow autodiff

w1, w2 = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape:
    z = f(w1, w2)

gradients = tape.gradient(z, [w1, w2])

gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [26]:
# gradient is automatically ereased when gradient is called
##--> tape.gradient(z, w1)

# another way is to set persistent=True so the gradient is over ereased, but 
# need to remember to delete the tape
with tf.GradientTape(persistent=True) as tape:
    z = f(w1, w2)
    
g1 = tape.gradient(z, [w1])
g2 = tape.gradient(z, [w2])

g1, g2

del tape

In [27]:
## Normally only tracks variables. but can also be set to track constant.
## this can be useful for some case. for example, implement a regularisation loss that 
## panelises activations that very a lot when inputs vary little
c1, c2 = tf.constant(5.), tf.constant(3.)
with tf.GradientTape() as tape:
    tape.watch(c1)
    tape.watch(c2)
    z = f(c1, c2)
    
gradients = tape.gradient(z, [c1, c2])
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

## Custom Training Loops

In [28]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
housing = fetch_california_housing()

keras.backend.clear_session()

# test_size is default to 0.25
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

X_train = StandardScaler().fit_transform(X_train)
X_valid = StandardScaler().fit_transform(X_valid)
X_test = StandardScaler().fit_transform(X_test)

In [57]:
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input


keras.backend.clear_session()
l2_reg = keras.regularizers.l2(1.0)


model = keras.models.Sequential([
    Dense(16, input_shape = X_train.shape[1:] , activation = 'relu'),
    Dense(1)
])

model.summary()
# how we would compile and run the model
model.compile(optimizer="adam", loss = 'mse', metrics=['mse'])
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=40)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                144       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 161
Trainable params: 161
Non-trainable params: 0
_________________________________________________________________
Epoch 1/40
363/363 [==============================] - 0s 802us/step - loss: 2.3630 - mse: 2.3630 - val_loss: 0.8399 - val_mse: 0.8399
Epoch 2/40
363/363 [==============================] - 0s 595us/step - loss: 0.7489 - mse: 0.7489 - val_loss: 0.5844 - val_mse: 0.5844
Epoch 3/40
363/363 [==============================] - 0s 595us/step - loss: 0.5571 - mse: 0.5571 - val_loss: 0.5690 - val_mse: 0.5690
Epoch 4/40
363/363 [==============================] - 0s 612us/step - loss: 0.4744 - mse: 0.4744 - val_loss: 0.5726 - 

In [58]:
import numpy as np
def random_batch(X, y, batch_size = 30):
    idx = np.random.randint(len(X), size = batch_size)
    return X[idx], y[idx]

def print_status_bar(iteration, total, loss, metrics = None):
    metrics = "-".join([f"{m.name}: {m.result()}"
                        for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print("\r{}/{} - ".format(iteration, total) + metrics, end = end)

In [62]:
keras.backend.clear_session()

n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = keras.optimizers.Adam(lr = 0.01)
loss_fn = keras.losses.mean_squared_error
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.MeanAbsoluteError()]

In [79]:
# 2 loops, top is for epochs, and second is for steps (see above)
for epoch in range(1, n_epochs + 1):
    print(f"Epoch: {epoch}/{n_epochs}")
    for step in range(1, n_steps + 1):
        X_batch, y_batch = random_batch(X_train, y_train)               # grab a random batch
        with tf.GradientTape() as tape:                                 # gradient tape created for loss
            y_pred = model(X_batch, training = True)                    # calculate prediction
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))        # calculate loss for the batch
            loss = tf.add_n([main_loss] + model.losses)                 # add other losses (for regularisation for example)
        gradients = tape.gradient(loss, model.trainable_variables)      # get gradients of all trainable variables
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))  # apply gradients, smart, as this can be optimiser specific such as learning rate, momentum, etc
        mean_loss(loss)             # calculate mean loss and meatrics (over the current epoch)
        for metric in metrics:
            metric(y_batch, y_pred)
    print_status_bar(len(y_train), len(y_train), mean_loss, metrics)
    for metric in [mean_loss] + metrics:
        metric.reset_states()

Epoch: 1/5
11610/11610 - mean: 1.3216079473495483-mean_absolute_error: 0.8897098302841187
Epoch: 2/5
11610/11610 - mean: 1.3708149194717407-mean_absolute_error: 0.910310685634613
Epoch: 3/5
11610/11610 - mean: 1.310929298400879-mean_absolute_error: 0.8860278129577637
Epoch: 4/5
11610/11610 - mean: 1.3164016008377075-mean_absolute_error: 0.8873282670974731
Epoch: 5/5
11610/11610 - mean: 1.328629970550537-mean_absolute_error: 0.8908034563064575


In [101]:
!git add 9.1_auto_diff.ipynb
!git commit -m "initial commit"
!git push

The file will have its original line endings in your working directory


[master 37e6f26] initial commit
 1 file changed, 56 insertions(+), 5 deletions(-)


To https://github.com/auslei/python.git
   3390cbd..37e6f26  master -> master


In [95]:
a = tf.constant(1., dtype = "float32")
b = tf.constant(2., dtype = "float32")

[a] + [b]

[<tf.Tensor: shape=(), dtype=float32, numpy=1.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.0>]

## Tensorflow fuction and Graph

In general TF function is much faster than vanila function as it is graph optimised.

**note**
if calling tf function with numeric value, a graph will be generated for each value, it can be extremely expensive. when passing in only tensors, the tensor will be reused.

In [96]:
def cube(x):
    return x**3

tf_cube = tf.function(cube)

In [98]:
%%timeit
cube(2)

173 ns ± 0.601 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [99]:
%%timeit
tf_cube(2)

93.7 µs ± 492 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [102]:
tf.autograph.to_code(tf_cube.python_function)

"def tf__cube(x):\n    with ag__.FunctionScope('cube', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:\n        do_return = False\n        retval_ = ag__.UndefinedReturnValue()\n        try:\n            do_return = True\n            retval_ = (ag__.ld(x) ** 3)\n        except:\n            do_return = False\n            raise\n        return fscope.ret(retval_, do_return)\n"